## Imports

In [1]:
import time

import numpy as np
import cv2 as cv
import os
import glob
import matplotlib.pyplot as plt
from cv2 import cvtColor
from numpy.random import uniform
import pdb
from tqdm import tqdm

import utils

In [2]:
game_tables = utils.extract_game(utils.get_image_paths("antrenare",1))

100%|██████████| 50/50 [00:12<00:00,  3.87it/s]


In [3]:
relevant_game_tables = utils.extract_relevant_game(game_tables)

100%|██████████| 50/50 [00:00<00:00, 114.33it/s]


In [4]:
utils.show_images_cv(relevant_game_tables,fx=0.3,fy=0.3)

In [5]:
delimited_tables = utils.draw_lines(relevant_game_tables)
utils.show_images_cv(delimited_tables,fx=0.3,fy=0.3)

100%|██████████| 50/50 [00:00<00:00, 931.68it/s]


In [5]:
utils.store_template_numbers1(10)
utils.store_template_numbers2(8)

100%|██████████| 1/1 [00:00<00:00, 87.28it/s]
